In [1]:
import numcodecs
import zarr
import os

In [2]:
%load_ext autoreload

In [3]:
%autoreload 2

In [4]:
import h5py

In [5]:
import cosilico_py.preprocessing.platform_helpers.x10 as x10
from cosilico_py.config import get_config

In [6]:
import typer
from pathlib import Path
import json

In [7]:
from dotenv import load_dotenv
from supabase import create_client

In [8]:
load_dotenv(dotenv_path='../../.env')
load_dotenv(dotenv_path='../../.env.test')

True

In [9]:
from cosilico_py.models import X10XeniumInput, DirectoryEntity, ExperimentViewSetting
from cosilico_py.client.client import CosilicoClient
from cosilico_py.config import get_config

In [12]:
def save_config(email, password):
    app_dir = typer.get_app_dir('cosilico_py')
    config_path: Path = Path(app_dir) / "config.json"
    d = {
        'api_url': os.environ.get('API_URL'),
        'anon_key': os.environ.get('ANON_KEY'),
#         'cache_dir': str((Path(app_dir) / 'cache').absolute()),
        'cache_dir': '~/Downloads/temp_cache',
        'email': email,
        'password': password,
        'preprocessing': {
            'layer': {
                'cells_max_vert_map': {
                    1: 64,
                    2: 64,
                    4: 64,
                    8: 64,
                    16: 64,
                    32: 64,
                    64: 64,
                    128: 64,
                    256: 64,
                    512: 64,
                    1024: 64,
                    2048: 64,
                    4096: 64,
                    8192: 8,
                    16384: 8,
                    32768: 8,
                    65536: 8,
                    131072: 8,
                    262144: 8,
                    524288: 8,
                    1048576: 8,
                },
                'cells_downsample_map': {
                    1: -1,
                    2: -1,
                    4: -1,
                    8: -1,
                    16: -1,
                    32: -1,
                    64: -1,
                    128: -1,
                    256: -1,
                    512: -1,
                    1024: -1,
                    2048: -1,
                    4096: -1,
                    8192: 100_000,
                    16384: 100_000,
                    32768: 100_000,
                    65536: 100_000,
                    131072: 100_000,
                    262144: 100_000,
                    524288: 100_000,
                    1048576: 100_000,
                },
                'cells_object_type_map': {
                    1: 'polygon',
                    2: 'polygon',
                    4: 'polygon',
                    8: 'polygon',
                    16: 'polygon',
                    32: 'polygon',
                    64: 'polygon',
                    128: 'polygon',
                    256: 'polygon',
                    512: 'polygon',
                    1024: 'polygon',
                    2048: 'polygon',
                    4096: 'polygon',
                    8192: 'polygon',
                    16384: 'polygon',
                    32768: 'polygon',
                    65536: 'polygon',
                    131072: 'polygon',
                    262144: 'polygon',
                    524288: 'polygon',
                    1048576: 'polygon',
                }
            }
        }

    }

    Path(d['cache_dir']).mkdir(parents=True, exist_ok=True)


    json.dump(d, open(config_path, 'w'))

def autocreate_client(email, password):
    save_config(email, password)
    client = CosilicoClient()
    client.sign_in()
    return client
    
    

In [11]:
# this is for doing the admin only
supabase = create_client(os.environ.get('API_URL'), os.environ.get('ANON_KEY'))
supabase

In [12]:
response = supabase.auth.sign_up(
    {
        "email": os.environ.get('TEST_ADMIN_EMAIL'),
        "password": os.environ.get('TEST_ADMIN_PASSWORD'),
        'options': {
            'data': {
                'name': os.environ.get('TEST_ADMIN_NAME'),
            }
        }
    }
)
response

AuthApiError: User already registered

In [12]:
## change to admin in terminal

In [ ]:
client = autocreate_client(os.environ.get('TEST_ADMIN_EMAIL'), os.environ.get('TEST_ADMIN_PASSWORD'))

In [ ]:
client.create_user(os.environ.get('TEST_USER1_EMAIL'), os.environ.get('TEST_USER1_PASSWORD'), os.environ.get('TEST_USER1_NAME'))
client.create_user(os.environ.get('TEST_USER2_EMAIL'), os.environ.get('TEST_USER2_PASSWORD'), os.environ.get('TEST_USER2_NAME'))


In [27]:
client = autocreate_client(os.environ.get('TEST_USER1_EMAIL'), os.environ.get('TEST_USER1_PASSWORD'))

Sign-in successful. 💥

In [14]:
client.create_directory('/project_a/subproject_a/zzz', permission='rw')

Directory already exists at /project_a/subproject_a/zzz

In [15]:
client.display_experiments()

Permission level 🔑: read write delete

└── 📁 project_a

└── 📁 subproject_a

└── 📁 zzz

└── 🔬 Xenium Prime + 100g Custom-Breast_Cancer

In [16]:
x_input = X10XeniumInput(
    cellranger_outs='/Users/erikstorrs/Downloads/Xenium_Prime_Breast_Cancer_FFPE_outs',
    bbox=(25000, 28000, 25000, 28000),
    to_uint8=True
)
x_input

X10XeniumInput(name=None, bbox=[25000, 28000, 25000, 28000], verbose=True, platform=<PlatformEnum.x10_xenium: '10X Xenium'>, cellranger_outs=PosixPath('/Users/erikstorrs/Downloads/Xenium_Prime_Breast_Cancer_FFPE_outs'), to_uint8=True)

In [ ]:
bundle = client.create_experiment(x_input)

In [18]:
client.upload_experiment(bundle, '/project_a/subproject_a/zzz')

Experiment Xenium Prime + 100g Custom-Breast_Cancer successfuly uploaded 🔬!

In [16]:
client.display_experiments()

Permission level 🔑: read write delete

└── 📁 project_a

└── 📁 subproject_a

└── 📁 zzz

└── 🔬 Xenium Prime + 100g Custom-Breast_Cancer

In [28]:
experiment = client.get_experiment('/project_a/subproject_a/zzz/Xenium Prime + 100g Custom-Breast_Cancer')
experiment

Xenium Prime + 100g Custom-Breast_Cancer - Xenium Morphology not in cache. Downloading... ⬇

filename cda6c46a-6f0c-47f4-8ce6-cf880d82f127.zarr.zip


RuntimeError: Download failed: 404 - <?xml version="1.0" encoding="UTF-8"?>
<Error><Code>NoSuchKey</Code><Message>The specified key does not exist.</Message><Key>cda6c46a-6f0c-47f4-8ce6-cf880d82f127.zarr.zip</Key><RequestId>4R4AR3SJ09M33ZMF</RequestId><HostId>QclL+O8fQVCSiXKEXawKrTE8lSkd9RDEUdIAAjFuaOQ/56HE8x7rfjRdUtvX/bilQjCX2mBOg0E=</HostId></Error>

In [91]:
# response = client.supabase.functions.invoke(
#     "get-directory-permissions",
#     invoke_options={
#         "body": {
#             'directory_ids': ['cc8d87ed-c728-46fe-aa81-dfa57c919151', '4fa3cf39-9039-4b68-bc20-e475a719e334']
#         },
#     },
# )
# body = json.loads(response.decode())
# body

In [40]:
from anytree.importer import DictImporter
from anytree import RenderTree
importer = DictImporter()

In [41]:
data = {
    'name': 'root',
    'id': 'lsdkfj',
    'p'
    'children': [
        {
            'name': 'sub0',
            'id': 'aldskfj'
            'children': [
                {
                    'a': 'sub0A',
                    'b': 'foo'
                },
                {
                    'a': 'sub0B'
                }
            ]
        },
        {
            'a': 'sub1'
        }
    ]
}

In [42]:
root = importer.import_(data)

In [43]:
print(RenderTree(root))

AnyNode(a='root')
├── AnyNode(a='sub0')
│   ├── AnyNode(a='sub0A', b='foo')
│   └── AnyNode(a='sub0B')
└── AnyNode(a='sub1')

In [45]:
node = None
for pre, _, node in RenderTree(root):
    print("%s%s" % (pre, node.name))

AttributeError: 'AnyNode' object has no attribute 'name'

In [46]:
node.a

'root'

In [21]:
# response = cc.supabase.functions.invoke(
#     "generate-upload-url",
#     invoke_options={
#         "body": {"filename": "lllll.zarr.zip"},
#     },
# )
# body = json.loads(response.decode())
# signed_url = body['url']
# signed_url

In [22]:
# import requests
# file_path = "/Users/erikstorrs/Library/Application Support/cosilico_py/cache/24eb1d9a1b3b4aa29dc653fd9deeb850.zarr.zip"

# with open(file_path, "rb") as f:
#     response = requests.put(
#         signed_url,
#         data=f,
#         headers={
#             "Content-Type": "application/zip"
#         }
#     )
# response

In [23]:
# response = cc.supabase.functions.invoke(
#     "generate-download-url",
#     invoke_options={
#         "body": {"filename": "lllll.zarr.zip"},
#     },
# )
# body = json.loads(response.decode())
# download_url = body['url']
# download_url

In [24]:
# output_path = "/Users/erikstorrs/Downloads/test.zarr.zip"

# # Download the file
# response = requests.get(download_url)

In [25]:
# with open(output_path, "wb") as f:
#     f.write(response.content)

In [32]:
x_input = X10XeniumInput(
    cellranger_outs='/Users/erikstorrs/Downloads/Xenium_Prime_Breast_Cancer_FFPE_outs',
    bbox=(25000, 28000, 25000, 28000), 
)
x_input

X10XeniumInput(name=None, bbox=[25000, 28000, 25000, 28000], verbose=True, platform=<PlatformEnum.x10_xenium: '10X Xenium'>, cellranger_outs=PosixPath('/Users/erikstorrs/Downloads/Xenium_Prime_Breast_Cancer_FFPE_outs'), to_uint8=False)

In [33]:
bundle = client.create_experiment(x_input)

Loading xenium experiment from /Users/erikstorrs/Downloads/Xenium_Prime_Breast_Cancer_FFPE_outs

Loading xenium morphology image from 
/Users/erikstorrs/Downloads/Xenium_Prime_Breast_Cancer_FFPE_outs/morphology_focus/morphology_focus_0002.ome.tif

<tifffile.TiffFile 'morphology_focus_0002.ome.tif'> OME series cannot read multi-file pyramids


Loading xenium transcripts from 
/Users/erikstorrs/Downloads/Xenium_Prime_Breast_Cancer_FFPE_outs/transcripts.parquet

Loading xenium transcript metadata for ['qv']

/Users/erikstorrs/Documents/viewer/cosilico-py/src/cosilico_py/preprocessing/core/tiling.py:28: RuntimeWarning: divide by zero encountered in remainder
  groups = np.arange(num_feats) % group_size  # Vectorized operation


Loading xenium cell boundaries from 
/Users/erikstorrs/Downloads/Xenium_Prime_Breast_Cancer_FFPE_outs/cell_boundaries.parquet

Loading xenium cell transcript counts 
/Users/erikstorrs/Downloads/Xenium_Prime_Breast_Cancer_FFPE_outs/cell_feature_matrix.h5

In [ ]:
client.create_experiment(x_input)

In [59]:
bundle.layers[0].local_path

PosixPath('/Users/erikstorrs/Library/Application Support/cosilico_py/cache/24eb1d9a1b3b4aa29dc653fd9deeb850.zarr.zip')

In [2]:
from ome_types import from_xml
import tifffile
import dask.array as da

In [19]:
image

ZarrTiffStore

In [20]:
ome_model

OME(
   plates=[{'id': 'Plate:1', 'well_origin_x': -0.0, 'well_origin_x_unit': <UnitsLength.MICROMETER: 'µm'>, 'well_origin_y': -0.0, 'well_origin_y_unit': <UnitsLength.MICROMETER: 'µm'>}],
   instruments=[{'microscope': {'manufacturer': '10x Genomics', 'model': 'Xenium', 'kind': 'microscope'}, 'id': 'Instrument:1'}],
   images=[<1 field_type>],
   uuid='urn:uuid:6f6e3c8f-db12-11ed-a5c7-0644c1d5d2e3',
   creator='tifffile.py 2021.4.8',
)

In [63]:
# ome_tiff_path = '/Users/erikstorrs/Downloads/Xenium_V1_FFPE_Human_Brain_Healthy_With_Addon_outs/morphology_focus.ome.tif'
ome_tiff_path = '/Users/erikstorrs/Downloads/Xenium_V1_FFPE_Human_Brain_Healthy_With_Addon_outs/morphology.ome.tif'
# ome_tiff_path = '/Users/erikstorrs/Downloads/Xenium_Prime_Breast_Cancer_FFPE_outs/morphology_focus/morphology_focus_0002.ome.tif'
# ome_tiff_path = '/Users/erikstorrs/Downloads/Xenium_V1_Human_Ductal_Adenocarcinoma_FFPE_outs/morphology_focus/morphology_focus_0000.ome.tif'
ome_metadata = tifffile.TiffFile(ome_tiff_path).ome_metadata
ome_model = from_xml(ome_metadata)

with tifffile.TiffFile(ome_tiff_path) as tif:
#     z = tif.aszarr()
#     zz = tif.series[0].aszarr()
    z = tif.series[0].pages[0].aszarr()
    image = da.from_zarr(z)
#     image = da.from_zarr(zz)
#     image = da.from_zarr(image, chunks=(1, 2048, 2048))

In [64]:
ome_model.images[0]

Image(
   id='Image:0',
   name='Image0',
   instrument_ref={'id': 'Instrument:1'},
   pixels={'channels': [{'id': 'Channel:0:0', 'name': 'DAPI', 'samples_per_pixel': 1}], 'tiff_data_blocks': [{'plane_count': 13}], 'id': 'Pixels:0', 'dimension_order': <Pixels_DimensionOrder.XYZCT: 'XYZCT'>, 'type': <PixelType.UINT16: 'uint16'>, 'size_x': 36955, 'size_y': 27282, 'size_z': 13, 'size_c': 1, 'size_t': 1, 'physical_size_x': 0.2125, 'physical_size_y': 0.2125, 'physical_size_z': 3.0},
)

In [65]:
for s in tif.series:
    print(len(s.pages), s)
    for p in s.pages:
        print(p)

13 TiffPageSeries 0  'Image0'  13x27282x36955  uint16  ZYX  ome  8 Levels  13 Pages
TiffPage 0 @16  27282x36955  uint16  minisblack tiled jpeg2000  ome
TiffFrame 1 @13939344  27282x36955  uint16
TiffFrame 2 @146243052  27282x36955  uint16
TiffFrame 3 @394109272  27282x36955  uint16
TiffFrame 4 @697471996  27282x36955  uint16
TiffFrame 5 @998401786  27282x36955  uint16
TiffFrame 6 @1298411118  27282x36955  uint16
TiffFrame 7 @1597568094  27282x36955  uint16
TiffFrame 8 @1896267556  27282x36955  uint16
TiffFrame 9 @2195048900  27282x36955  uint16
TiffFrame 10 @2496114066  27282x36955  uint16
TiffFrame 11 @2786268448  27282x36955  uint16
TiffFrame 12 @2959810478  27282x36955  uint16


In [49]:
for p in tif.pages:
    print(p)

TiffPage 0 @16  74945x51265  uint16  minisblack tiled jpeg2000  ome


In [59]:
image

dask.array<from-zarr, shape=(27282, 36955), dtype=uint16, chunksize=(1024, 1024), chunktype=numpy.ndarray>

In [43]:
s = tif.series[0]
s

<tifffile.TiffPageSeries 0 ome>

In [46]:
s.aszarr()

AssertionError: 

In [6]:
import pandas as pd

In [8]:
df = pd.read_csv('/Users/erikstorrs/Downloads/Xenium_Prime_Breast_Cancer_FFPE_outs/metrics_summary.csv')
df

,run_name,cassette_name,region_name,panel_name,panel_design_id,predesigned_panel_id,region_area,total_cell_area,total_high_quality_decoded_transcripts,fraction_transcripts_decoded_q20,...,fraction_of_ambiguous_cell_mask_pixels,fraction_of_nucleus_polygons_removed,fraction_of_cell_polygons_removed,fraction_of_nuclei_without_cell,number_of_cell_non_simple_polygons,number_of_cell_multi_polygons,number_of_nucleus_non_simple_polygons,number_of_nucleus_multi_polygons,segmented_cell_imported_frac,segmented_cell_imported_count
0,Xenium Prime + 100g Custom,NaN,Breast_Cancer,Xenium Human 5K with Cell Typing + HPV-16/18 a...,NaN,hAtlas_v1.1,1.551598e+08,4.553694e+07,80158637,0.86385,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
df.to_json()

'{"run_name":{"0":"Xenium Prime + 100g Custom"},"cassette_name":{"0":null},"region_name":{"0":"Breast_Cancer"},"panel_name":{"0":"Xenium Human 5K with Cell Typing + HPV-16\\/18 and SNV Add-on"},"panel_design_id":{"0":null},"predesigned_panel_id":{"0":"hAtlas_v1.1"},"region_area":{"0":155159780.8498440087},"total_cell_area":{"0":45536938.5737275034},"total_high_quality_decoded_transcripts":{"0":80158637},"fraction_transcripts_decoded_q20":{"0":0.8638496091},"fraction_predesigned_transcripts_decoded_q20":{"0":0.8662174711},"fraction_custom_transcripts_decoded_q20":{"0":0.8272998337},"nuclear_transcripts_per_100um2":{"0":196.2993157169},"decoded_transcripts_per_100um2":{"0":157.1654402812},"adjusted_negative_control_probe_rate":{"0":0.0013769231},"adjusted_negative_control_codeword_rate":{"0":0.0002238943},"adjusted_genomic_control_probe_rate":{"0":0.0336418666},"negative_control_probe_counts_per_control_per_cell":{"0":0.00002918},"genomic_control_probe_counts_per_control_per_cell":{"0":0

In [11]:
from dateutil import parser

dt = parser.isoparse("2024-06-13T21:22:41Z")
dt

datetime.datetime(2024, 6, 13, 21, 22, 41, tzinfo=tzutc())

In [2]:
print('[bold red]Sign in attempt failed.[/bold red]')

Sign in attempt failed.

In [5]:
print('[green]Sign in successfull.[/green]')

Sign in successfull.

In [2]:
adata = x10.read_10x_h5('/Users/erikstorrs/Downloads/Xenium_Prime_Breast_Cancer_FFPE_outs/cell_feature_matrix.h5')
adata.obs

""
aaaaaohf-1
aaaabkoj-1
aaaafefl-1
aaaahfjm-1
aaaahjao-1
...
oimkalkm-1
oimkankb-1
oimkapok-1
oimkbcmb-1


In [9]:
import pandas as pd
s = pd.Series(pd.Categorical(['a', 'a', 'b', 'd']))
s.cat.categories.to_list()

['a', 'b', 'd']

In [2]:
import pandas as pd
dtype_dict = {
        "feature_name": "category",
        "is_gene": "bool",
        "codeword_category": "category",
        "x_location": "float32",
        "y_location": "float32",
        "transcript_id": "uint64",
        "qv": "float32",
    }
source = pd.read_parquet(
    "/Users/erikstorrs/Downloads/Xenium_Prime_Breast_Cancer_FFPE_outs/transcripts.parquet",
    columns=["feature_name", "is_gene", "codeword_category", "x_location", "y_location", "transcript_id", "qv"],  # Read only needed columns
    engine="pyarrow"  # Ensure efficient Parquet reading
).astype(dtype_dict)  # Convert data types immediately

In [3]:
adata.var

,gene_ids,gene_names
0,ENSG00000166535,A2ML1
1,ENSG00000127837,AAMP
2,ENSG00000131043,AAR2
3,ENSG00000266967,AARSD1
4,ENSG00000183044,ABAT
...,...,...
9470,DeprecatedCodeword_18632,DeprecatedCodeword_18632
9471,DeprecatedCodeword_18637,DeprecatedCodeword_18637
9472,DeprecatedCodeword_18639,DeprecatedCodeword_18639
9473,DeprecatedCodeword_18646,DeprecatedCodeword_18646


In [4]:
adata.X.shape

(699110, 9475)

In [17]:
import zarr
import numpy as np

In [43]:
output_filepath = '/Users/erikstorrs/Downloads/tempjj.zarr.zip'
store = zarr.storage.ZipStore(output_filepath, mode='w')
root = zarr.group(store=store, overwrite=True)
g = root.create_group("g1")
# x = np.asarray(['hello', 'aaaaaaaaddddddddeeeeeeeeeeaaaaaaaaddddddddeeeeeeeeeeaaaaaaaaddddddddeeeeeeeeeeaaaaaaaaddddddddeeeeeeeeeeaaaaaaaaddddddddeeeeeeeeee', 'a', 'b', ',c']).astype(str)
# z = g.create_array("testd", shape=(len(x),), chunks=(len(x),), dtype='U50')
# z[:] = x

x = np.asarray([1,2,3], dtype=np.float32)
z = g.create_array("testa", shape=(len(x),), chunks=(len(x),), dtype='float32')
z[:] = x

root.tree()


/
└── g1
    └── testa (3,) float32

In [44]:
store.close()

In [45]:
store = zarr.storage.ZipStore(output_filepath, mode='w')
root = zarr.group(store=store)
print(root.tree())
store.close()

/

In [22]:
store = zarr.storage.ZipStore(output_filepath, mode='r')
root = zarr.group(store=store)
root['/g1/testd']

ValueError: Store is read-only but mode is 'r+'